In [858]:
import pandas as pd
import numpy as np
import sc2reader

In [859]:
rep_folder = "data/s2client_blizz_replays/"
rep_names = [rep_name for root_dir, rep_dir, rep_name in os.walk(rep_folder)][0]
rep_paths = [rep_folder + rep_name for rep_name in rep_names]

In [860]:
def loadReplay(rep_path):
    replay = sc2reader.load_replay(rep_path, load_level=4)
    return replay

In [ ]:
%%time
data = [loadReplay(rep_path) for rep_path in rep_paths[:10000]]

In [819]:
def getPlayersData(replay):
    player_data = [player.__dict__ for player in replay.players]
    game_length = replay.length.seconds
    return [makeDatapoint(player, game_length) for player in player_data]

In [837]:
def makeDatapoint(player, game_length):
    result = player['result']
    race = player['attribute_data']['Race']
    events = len(player['events'])
    league_index = player['highest_league']
    return race, league_index, events, result, game_length, 

In [838]:
playerPairs = [getPlayersData(replay) for replay in data]
raw_dataset = [player for playerPair in playerPairs for player in playerPair]

In [856]:
cols = ['Race', 'LeagueIndex', 'Actions', 'GameResult', 'GameLength']
df = pd.DataFrame(raw_dataset, columns=cols)
df['APM'] = df['Actions'] / (df['GameLength'] / 60)
df

,Race,LeagueIndex,Actions,GameResult,GameLength,APM
0,Protoss,6,6524,Loss,932,420.000000
1,Zerg,6,7347,Win,932,472.982833
2,Terran,5,1341,Win,324,248.333333
3,Terran,5,961,Loss,324,177.962963
4,Random,5,1460,Loss,667,131.334333
5,Zerg,0,2046,Win,667,184.047976
6,Zerg,3,2808,Win,775,217.393548
7,Terran,3,2768,Loss,775,214.296774
8,Zerg,5,1791,Win,537,200.111732
9,Zerg,5,2391,Loss,537,267.150838


In [857]:
apm_for_winners = df.groupby('GameResult').median().reset_index()[['GameResult', 'APM']]
apm_for_winners

,GameResult,APM
0,Loss,217.850083
1,Win,231.589944
